In [3]:
"""
使用 LangGraph Supervisor 类库搭建具有数学智能体和研究智能体的主管架构
"""

import os

from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langchain.agents import create_agent

from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(
    model="qwen3-max-2026-01-23",
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    temperature=0,
)


def add(a: float, b: float) -> float:
    """两个数字相加"""
    return a + b


def multiply(a: float, b: float) -> float:
    """两个数字相乘"""
    return a * b


def web_search(query: str) -> str:
    """在网络上搜索信息"""
    return (
        "以下是 FAANG 公司 2024 年的员工人数：\n"
        "1. **Facebook (Meta)**: 67,317 名员工。"
        "2. **Apple**: 164,000 名员工。"
        "3. **Amazon**: 1,551,000 名员工。"
        "4. **Netflix**: 14,000 名员工。"
        "5. **Google (Alphabet)**: 181,269 名员工。"
    )


math_agent = create_agent(
    model=model,
    tools=[add, multiply],
    name="math_expert",
    system_prompt="你是一位数学专家，始终使用一个工具。",
)

research_agent = create_agent(
    model=model,
    tools=[web_search],
    name="research_expert",
    system_prompt="你是一位世界一流的研究专家，可以进行网络搜索，不要做任何数学运算。",
)

workflow = create_supervisor(
    agents=[research_agent, math_agent],
    model=model,
    prompt="你是一位团队主管，管理一位研究专家和一位数学专家。研究专家能够利用网络搜索工具进行查询。",
)

app = workflow.compile()

result = app.invoke(
    {
        "messages": [
            {"role": "user", "content": "2024 年 FAANG 公司的总员工人数是多少？"}
        ]
    }
)

print(result["messages"][-1].content)

截至 2024 年，FAANG 公司的总员工人数为 **1,977,586** 人。具体分布如下：

- **Meta (Facebook)**: 67,317 人  
- **Apple**: 164,000 人  
- **Amazon**: 1,551,000 人  
- **Netflix**: 14,000 人  
- **Google (Alphabet)**: 181,269 人  

如需进一步分析或数据来源，请随时告知！
